# Rust 基本文法

## 関数

関数とは何かしらの値を受け取り、それらを処理した結果の値を返すものである

### fn
関数を定義するためには `fn` キーワードを用いる

In [2]:
/// add: i32型 の引数 a, b を受け取り、それらを加算した結果の i32型 の値を返す関数
fn add(a: i32, b: i32) -> i32 {
    return a + b; // return 文で戻り値を指定
}

/// sub: i32型 の引数 a, b を受け取り、それらを減算した結果の i32型 の値を返す関数
fn sub(a: i32, b: i32) -> i32 {
    a - b // Rust は最後に評価した式は自動的に return される (; を付けると文になってしまうため注意)
}

fn main() {
    let a = 10;
    let b = 20;
    let added = add(a, b); // => 30
    let subbed = sub(a, b); // => -10
    
    println!("{}, {}", added, subbed);
}
main();

30, -10


### クロージャ
Rustにおけるクロージャとは、変数に束縛できたり、関数の引数として渡すことのできる無名関数のことである

クロージャはその呼び出し元のスコープにある変数をキャプチャすることもできる

厳密に言うと、無名関数の中で束縛していない変数のことを自由変数と言い、自由変数をまとめた環境を無名関数のスコープ内に閉じこめたものをクロージャと呼ぶ

クロージャは `||` で定義し、引数があれば `|param1: Type1, param2: Type2|` のように `||` の間に入れる

その後に `{}` で本体を記述するが、本体が式1つだけであれば `{}` を省略することができる

次のコードは関数とそれと同じ振る舞いをするクロージャの例である

```rust
fn  add_one_v1   (x: u32) -> u32 { x + 1 }
let add_one_v2 = |x: u32| -> u32 { x + 1 };
let add_one_v3 = |x|             { x + 1 };
let add_one_v4 = |x|               x + 1  ;
```

クロージャはそれぞれ独自の型を持っており、`Fn`, `FnMut`, `FnOnce` トレイトのいずれかのインスタンスとなっている

それぞれ `&self`, `&mut self`, `self` を内部的に引数として受け取っているかどうかの違いがある

また、`Fn` は `FnMut` を、`FnMut` は `FnOnce` を継承している

#### Fn と fn
`Fn` トレイトと `fn` キーワードは別ものである

`fn` は関数定義で使うが、同時に型でもある

そして、`fn` のことを **関数ポインタ** と呼ぶ

`fn` は `Fn` のインスタンスであるため、`FnMut`, `FnOnce` のインスタンスでもある、という関係になっている

なお、クロージャはトレイトで実装されているため、コンパイル時にサイズがわからない

そのため、例えば、次のようにクロージャを返す関数はエラーになる

In [2]:
/// クロージャを返す関数
// => サイズ不明のためコンパイルエラー
fn get_closure() -> Fn(i32) -> i32 {
    |x| x + 1
}

Error: trait objects without an explicit `dyn` are deprecated

Error: return type cannot have an unboxed trait object

In [3]:
// 上記エラーを解決するためには、Box型で包んでヒープ領域にクロージャを保存するようにすれば良い
// これにより get_closure が返すのはトレイトではなく関数ポインタとなり、問題なくコンパイルできるようになる

fn get_closure() -> Box<Fn(i32) -> i32> {
    Box::new(|x| x + 1)
}

let increment = get_closure();
println!("{}", increment(1)); // => 2

2


#### キャプチャ
クロージャではスコープ内の変数を自動的にキャプチャして参照することができる

ここで、クロージャのキャプチャは、所有権の移動（ムーブセマンティクス）ではなく参照のコピー（借用）であるという点に注意が必要である

詳しくはスレッドの項にて解説する

In [5]:
for x in 0..10 {
    let get_next = || x + 1; // x を参照可能
    println!("{}", get_next());
}

1
2
3
4
5
6
7
8
9
10


()

### impl
Rustはオブジェクト指向言語ではないが、ある構造体に関係の深い関数を紐付けて定義することで、オブジェクト指向風の書き方をすることが可能

`impl` キーワードを使うことで、構造体にメソッドを加えることが可能である

In [3]:
// 自作型: Person構造体
struct Person {
    name: String,
    age: u32,
}

// Person構造体に関数を紐付けていく
impl Person {
    // impl ブロック内で定義する関数の第一引数には、構造体へのポインタを渡す
    // 戻り値に自分自身のポインタを指定することでメソッドチェーンを使うことができるようになる
    // ※ メソッドチェーン: person.say_hello().say_age()... のようにメソッドを連ねること
    fn say_hello(&self) -> &Self {
        println!("I am {}.", self.name);
        self
    }
    
    fn say_age(&self) -> &Self {
        println!("I am {} year(s) old.", self.age);
        self
    }
    
    // 構造体の値を変更したい場合は第一引数に mutable 構造体ポインタを渡す
    fn set_age(&mut self, age: u32) -> &Self {
        self.age = age;
        self
    }
}

fn main() {
    let mut p = Person{
        name: String::from("John"),
        age: 0,
    };
    
    // impl で紐付けた関数は、オブジェクト指向のメソッドのような記述方法で呼び出すことができる
    // ※ 自動的に第一引数に対象の構造体のポインタが渡される
    p
        .set_age(10) // set_age(&p, 10) => p.age: 10
        .say_hello() // => I am John.
        .say_age(); // => I am 10 year(s) old.
}
main();

I am John.
I am 10 year(s) old.


## マクロ

Rustでは抽象構文木に直接影響を与えるような強力なマクロ機能を使うことができる

ここでは、関数のように扱えるマクロと、`#[derive]` attribute (定義の前の行に付与して該当定義を拡張する表現) によって実装を自動的に導出するマクロの2種類を解説する

### 標準ライブラリのマクロ
Rustではいくつかのマクロが標準で提供されている

- https://github.com/rust-lang/rust/blob/master/library/core/src/macros/mod.rs

マクロ呼び出しには `!` が付き、関数呼び出しと見た目で区別できるようになっている

なお、マクロ定義では慣例的に、Rustの構文と見た目が合うような括弧が使われる

例えば関数呼び出し的なマクロなら `()`、コードブロックを引数に取るようなマクロなら `{}` といった具合である

#### 文字列操作マクロ
- `format!`
    - フォーマット文字列と値から文字列を作成するマクロ
    - 文字列スライスを結合して新しい文字列を作成する際にも使われる
    - フォーマット文字列:
        - `{:?}`
            - std::fmt::Debug トレイトで定義
            - データの内部情報の文字列表現
            - 自作型で定義する場合は `#[derive(Debug)]` attribute で実装する
        - `{}`
            - std::fmt::Display トレイトで定義
            - ユーザに提示するためのシンプルな文字列
            - 自作型で定義する場合は impl キーワードで実装する必要がある
- concat!
    - リテラルを結合するマクロ

In [4]:
fn main() {
    let s = concat!("A", "b2", 3);
    println!("{}", s); // => Ab23
    
    let s = format!("{}-{:?}", "c", ("D", 5));
    println!("{}", s); // => c-("D", 5)
}
main();

Ab23
c-("D", 5)


#### データ出力用マクロ
- `print!`, `println!`
    - フォーマット結果を標準出力に出力するマクロ
    - `print!` は末尾改行なしで、`println!` は末尾改行ありで出力する
- `eprint!`, `eprintln!`
    - フォーマット結果を標準エラーに出力するマクロ
- `write!`, `writeln!`
    - バッファーにバイト列を出力するマクロ
- `dbg!`
    - 呼び出し元のファイル名、行数、式、式の値を標準エラーに出力するマクロ

In [5]:
use std::io::Write; // write, writeln マクロを使うため

fn main() {
    print!("Hello, ");
    println!("World!!");
    eprint!("{}: ", "Error");
    eprintln!("{}", "I'm error message");
    
    let mut bytes = Vec::new(); // バイト列書き込み用のベクタ
    write!(&mut bytes, "{}", "ABC"); // UTF-8バイト列で 65, 66, 67 書き込み
    writeln!(&mut bytes, "{}", "is 123"); // UTF-8バイト列で 32, 105, 115, 32, 49, 50, 51, 10(\n) 書き込み
    
    // デバッグ: フォーマット文字列は使えない
    dbg!(bytes);
}
main();

Hello, World!!


Error: I'm error message
[src/lib.rs:47] bytes = [
    65,
    66,
    67,
    105,
    115,
    32,
    49,
    50,
    51,
    10,
]


#### 異常終了用のマクロ
- `panic!`
    - プログラムを異常終了するマクロ

In [6]:
fn main() {
    panic!("it will panic");
}
main();

thread '<unnamed>' panicked at 'it will panic', src/lib.rs:37:5
stack backtrace:
   0: std::panicking::begin_panic
   1: __ImageBase
   2: run_user_code_5
   3: <unknown>
   4: <unknown>
   5: <unknown>
   6: <unknown>
   7: <unknown>
   8: <unknown>
   9: <unknown>
  10: <unknown>
  11: BaseThreadInitThunk
  12: RtlUserThreadStart
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 0xc0000005

#### ベクタを初期化するマクロ
- `vec!`
    - `Vec<T>` を手軽に初期化するマクロ
    - 配列と同じ記法になるように、慣例的に `[]` を使って呼び出す

#### プログラム外のリソースにアクセスするマクロ
- `file!`
    - 呼び出し元ファイル名を取得するマクロ
- `line!`
    - 呼び出し元ソースコードの行数を取得するマクロ
- `cfg!`
    - コンパイラから該当フラグが渡されていれば `true`, 渡されていなければ `false` を返すマクロ
- `env!`
    - コンパイル時の環境変数を取得するマクロ

In [7]:
println!("defined in file: {}", file!());
println!("defined on line: {}", line!());
println!("compiler defined unix flag: {}", cfg!(unix));
println!("CARGO_HOME: {}", env!("CARGO_HOME"));

defined in file: src/lib.rs
defined on line: 58
compiler defined unix flag: false
CARGO_HOME: C:\Users\user\.cargo


#### アサーション用のマクロ
- `assert!`, `debug_assert!`
    - 引数が `true` であることを表明するマクロ
- `assert_eq!`, `debug_assert_eq!`
    - 2つの引数が等しいことを表明するマクロ
- `assert_ne!`, `debug_assert_ne!`
    - 2つの引数が等しくないことを表明するマクロ

`debug_***` のマクロは、デバッグビルドの場合のみ実行され、リリースビルドの場合は無視されてコンパイルが成功する

In [8]:
fn main() {
    assert!(true);
    assert_eq!(1, 1);
    assert_ne!(1, 1);
    
    debug_assert!(false);
    debug_assert_eq!(1, 1);
    debug_assert_ne!(1, 1);
}
main();

thread '<unnamed>' panicked at 'assertion failed: `(left != right)`
  left: `1`,
 right: `1`', src/lib.rs:39:5
stack backtrace:
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 0xc0000005

### トレイトの導出
標準ライブラリのいくつかのトレイトは、自作の型に対する標準的な実装を自動的に導出 (**derive**) することができる

導出には `#[derive]` attribute マクロを用いる

In [9]:
// 一致比較可能な構造体 A を定義
// => Eq, PartialEq を derive する
#[derive(Eq, PartialEq)]
struct A(i32);

println!("{:?}", A(0) == A(1));


// 大小比較可能な構造体 B を定義
// => PartialEq, PartialOrd を derive する
#[derive(PartialEq, PartialOrd)]
struct B(f32);

println!("{:?}", B(1.0) > B(0.5));


// デバッグプリント {:?} が可能な構造体 C を定義
// => Debug を derive する
#[derive(Debug)]
struct C;

println!("{:?}", C);

false
true
C
